# Build prediction file 

Builds the prediction file as expected for the evaluation :
file : tab format with 3 columns : file_name, MeSH_Label, description 


(Prend en entrée la référence (classes-train-v2.txt) et l'hypothèse
(sortie-system). Les deux fichiers sont au format tabulaire en trois
colonnes : nom du fichier, nom simplifié du chapitre du MeSH, un
exemple issu du texte pour ce chapitre et ce fichier)

In [21]:
from os.path import isfile, isdir, join, basename
from os import listdir
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import collections
import math
from sklearn import metrics

IN_BRAT_DIR = '../../data/release/train2021/'
OUT_DIR = '../../data/release/train2021/'

IN_TRAIN_FILE = '../../data/work/classes-train-train.txt'
IN_DEV_FILE = '../../data/work/classes-train-val.txt'
IN_FR_MESH_FILE = '../../resources/terminologie_MeSH_DEFT2021.xlsx'

OUT_DF = '../../data/mesh_term_dataset.pkl'

assert isdir(IN_BRAT_DIR)
assert isfile(IN_TRAIN_FILE)
assert isfile(IN_FR_MESH_FILE)
assert isfile(IN_DEV_FILE)

In [15]:
# Read all Brat files in Brat dir and keep the MeSH corresponding labels

# first extract the set of labels
deft_dfs = {}
labels = set()
for source, f in zip(['DEFT-train', 'DEFT-val'], [IN_TRAIN_FILE, IN_DEV_FILE]):
    labels_df = pd.read_csv(f, sep='\t', header=None, names=['file', 'label', 'desc'])
    labels |= set(labels_df['label'].unique())
    files = labels_df['file'].unique()
    deft_dfs[source] = (labels_df, files)


    

{'ORL',
 'blessures',
 'cardiovasculaires',
 'chimiques',
 'digestif',
 'endocriniennes',
 'etatsosy',
 'femme',
 'genetique',
 'hemopathies',
 'homme',
 'immunitaire',
 'infections',
 'nerveux',
 'nutritionnelles',
 'oeil',
 'osteomusculaires',
 'parasitaires',
 'peau',
 'respiratoire',
 'stomatognathique',
 'tumeur',
 'virales'}

In [40]:
ENTITY_REGEX = re.compile('^(T\d+)\t([^ ]+) ([^\t]+)\t(.*)$')

data = []
# extract all ann_files from IN_BRAT_DIR
ann_files = [f for f in listdir(IN_BRAT_DIR) if isfile(join(IN_BRAT_DIR, f)) if f.endswith('.ann')]


with open("test.txt", "w+") as filout:
    for ann_file in ann_files:
        ann_path = join(IN_BRAT_DIR, ann_file)
        assert isfile(ann_path)
        # Read ann file
        with open(ann_path, 'r', encoding='utf-8') as f_in:
            lines = f_in.readlines()
    
        columns = ['term', 'label', 'source']

        # First pass -> extract all labels (i.e entity with Mesh-like tags)
        label_infos = {}
        other_infos = {}
        for line in lines:
            entity_match = ENTITY_REGEX.match(line.strip())
            if entity_match is not None:
                ann_id = entity_match.group(1)
                label = entity_match.group(2)
                offsets = entity_match.group(3)
                span = entity_match.group(4)
                if label in labels:
                    label_list = label_infos.get(offsets, [])
                    label_list.append((ann_id, label, offsets, span))
                    label_infos[offsets] = label_list
                    data.append([basename(ann_path), label, span])
                    that_line = [basename(ann_path), label, span]
                    that_line_txt = "\t".join(that_line) 
                    filout.write("{}\n".format(that_line_txt))

                    

